In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
batch_size = 32

base_model = MobileNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
pred = Dense(2, activation='softmax', use_bias=True)(x)

model = Model(inputs=base_model.input, outputs=pred)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

pred_list = []
test_acc = []



In [ ]:
for i in range(11,21):
    model_path = './results/check/check_%d' % i
    model.load_weights(model_path)

    test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

    test_generator = test_datagen.flow_from_directory('D:/Data/clearness/test',
                                                target_size = (224, 224),
                                                batch_size = batch_size,
                                                shuffle=False,
                                                class_mode = 'categorical')

    labels = test_generator.classes
    


    test_steps = test_generator.n//test_generator.batch_size
    test = model.evaluate_generator(generator=test_generator,
                                    steps=test_steps)

    test_acc.append(test)

    print(test)

In [ ]:
for i in range(11,21):
    model_path = './results/check/check_%d' % i
    model.load_weights(model_path)

    test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

    test_generator = test_datagen.flow_from_directory('D:/Data/clearness/test',
                                                target_size = (224, 224),
                                                batch_size = batch_size,
                                                shuffle=False,
                                                class_mode = 'categorical')

    labels = test_generator.classes
    


    test_steps = test_generator.n//test_generator.batch_size+1

    preds = model.predict_generator(generator=test_generator,
                                    steps=test_steps)

    pred_list.append(preds)
    print(preds[0])

In [ ]:
print(test_acc)

In [ ]:
pred_labels = []
for i in range(len(pred_list)):
    temp = pred_list[i]
    temp2 = np.argmax(temp, axis=1)
    pred_labels.append(temp2)

In [ ]:
wrongs = []
for i in range(len(pred_labels)):
    temp = np.logical_xor(labels, pred_labels[i])
    wrongs.append(temp)

In [ ]:
wrongs = np.array(wrongs)

In [ ]:
wrong_idx = []
for i in range(len(wrongs)):
    temp = np.where(wrongs[i]==True)
    wrong_idx.append(temp[0])

In [ ]:
file_names = {}
for i in range(len(wrong_idx)):
    file_names[i] = []
    for j in range(len(wrong_idx[i])):
        name = test_generator.filenames[wrong_idx[i][j]]
        file_names[i].append(name)

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg

In [ ]:
temp_names = 'D:/Data/clearness/test/' + file_names[0][0]
print(temp_names)

In [ ]:
if '/clear' in temp_names:
    print("!")

In [ ]:
temp_names = file_names[0]

In [ ]:
temp_names = file_names[0]
ucount = 0
for i in range(len(temp_names)):
    fullname = 'D:/Data/clearness/test/' + temp_names[i]
    if 'unclear' in fullname:
        img = mpimg.imread(fullname)
        plt.imshow(img)
        plt.show()
        ucount+=1

In [ ]:
temp_names = file_names[0]
ccount = 0
for i in range(len(temp_names)):
    fullname = 'D:/Data/clearness/test/' + temp_names[i]
    if 'test/clear' in fullname:
        img = mpimg.imread(fullname)
        plt.imshow(img)
        plt.show()
        ccount+=1

In [ ]:
print(ccount, ucount)